# Logistic Regression

* [강의출처](https://www.youtube.com/watch?v=HgPWRqtg254&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=8)
* [코드출처](https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-05_logistic_classification.ipynb)

## Module

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# For reproducibility
torch.manual_seed(42)

In [3]:
# data
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
x_train.shape, y_train.shape

(torch.Size([6, 2]), torch.Size([6, 1]))

In [5]:
# hypothesis

W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
hypothesis = 1 / (1 + torch.exp(-x_train.matmul(W) + b))

# same
# hypothesis = torch.sigmoid(x_train.matmul(W) + b)

hypothesis

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)

In [6]:
cost = torch.mean(-(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)))

## Full code

In [7]:
# data
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# initialize
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([W, b], lr=1)
nb_epochs = 1000

for epoch in range(1, nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = -(y_train * torch.log(hypothesis) + 
             (1 - y_train) * torch.log(1 - hypothesis)).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch  100/1000 Cost: 0.135644
Epoch  200/1000 Cost: 0.080964
Epoch  300/1000 Cost: 0.058062
Epoch  400/1000 Cost: 0.045398
Epoch  500/1000 Cost: 0.037327
Epoch  600/1000 Cost: 0.031720
Epoch  700/1000 Cost: 0.027592
Epoch  800/1000 Cost: 0.024422
Epoch  900/1000 Cost: 0.021911
Epoch 1000/1000 Cost: 0.019871


## Using Pytorch Method

In [8]:
class LogisticRegression(nn.Module):
    def __init__(self, input_shape, output_shape):
        super().__init__()
        self.linear = nn.Linear(input_shape, output_shape)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [9]:
# data
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# initialize
model = LogisticRegression(2, 1)
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(1, nb_epochs + 1):

    # Cost 계산
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch   10/100 Cost: 0.605884 Accuracy 66.67%
Epoch   20/100 Cost: 0.424702 Accuracy 83.33%
Epoch   30/100 Cost: 0.358176 Accuracy 83.33%
Epoch   40/100 Cost: 0.303257 Accuracy 83.33%
Epoch   50/100 Cost: 0.254213 Accuracy 83.33%
Epoch   60/100 Cost: 0.209952 Accuracy 100.00%
Epoch   70/100 Cost: 0.174402 Accuracy 100.00%
Epoch   80/100 Cost: 0.153294 Accuracy 100.00%
Epoch   90/100 Cost: 0.141469 Accuracy 100.00%
Epoch  100/100 Cost: 0.132029 Accuracy 100.00%


## Real Data

In [10]:
class LogisticRegression(nn.Module):
    def __init__(self, input_shape, output_shape):
        super().__init__()
        self.linear = nn.Linear(input_shape, output_shape)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

### data

In [11]:
data = pd.read_csv('./samples/diabetes.csv', header=None)

In [12]:
x_data = data.iloc[:, :-1].copy()
y_data = data.iloc[:, [-1]].copy()
x_data = (x_data - x_data.mean()) / x_data.std()

In [13]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1)

In [14]:
x_train = torch.FloatTensor(x_train.values)
y_train = torch.FloatTensor(y_train.values)
x_val = torch.FloatTensor(x_val.values)
y_val = torch.FloatTensor(y_val.values)

In [15]:
x_train.shape, x_val.shape

(torch.Size([691, 8]), torch.Size([77, 8]))

### init

In [16]:
# constants
input_size = x_train.shape[-1]
output_size = 1
nb_epochs = 100000
learning_rate = 0.0001

In [17]:
# model
model = LogisticRegression(input_size, output_size)

# optimizer
opt = optim.SGD(model.parameters(), lr=learning_rate)

### training

In [18]:
for epoch in range(1, nb_epochs + 1):

    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    opt.zero_grad()
    cost.backward()
    opt.step()

    # 100번마다 로그 출력
    if epoch % 10000 == 0:
        prediction = hypothesis >= 0.5
        accuracy = (prediction.float() == y_train).sum().item() / len(prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy: {:.6f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100
        ))

Epoch 10000/100000 Cost: 0.606298 Accuracy: 67.293777%
Epoch 20000/100000 Cost: 0.566297 Accuracy: 71.345876%
Epoch 30000/100000 Cost: 0.540983 Accuracy: 73.661360%
Epoch 40000/100000 Cost: 0.523960 Accuracy: 74.819103%
Epoch 50000/100000 Cost: 0.511979 Accuracy: 75.397974%
Epoch 60000/100000 Cost: 0.503252 Accuracy: 76.121563%
Epoch 70000/100000 Cost: 0.496726 Accuracy: 76.555716%
Epoch 80000/100000 Cost: 0.491740 Accuracy: 77.713459%
Epoch 90000/100000 Cost: 0.487867 Accuracy: 77.858177%
Epoch 100000/100000 Cost: 0.484815 Accuracy: 78.292330%


### test

In [20]:
prediction = model(x_val) >= 0.5
accuracy = (prediction.float() == y_val).sum().item() / len(prediction)
accuracy

0.7012987012987013